In [3]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [4]:
data = Dataset.load_builtin('ml-100k')
trainset,testset = train_test_split(data,test_size=.25, random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to C:\Users\admin/.surprise_data/ml-100k


In [5]:
algo = SVD(random_state=0)
algo.fit(trainset)

In [6]:
predictions = algo.test(testset)
print(len(predictions))
print(predictions[:5])

25000
[Prediction(uid='120', iid='282', r_ui=4.0, est=3.5114147666251547, details={'was_impossible': False}), Prediction(uid='882', iid='291', r_ui=4.0, est=3.573872419581491, details={'was_impossible': False}), Prediction(uid='535', iid='507', r_ui=5.0, est=4.033583485472447, details={'was_impossible': False}), Prediction(uid='697', iid='244', r_ui=5.0, est=3.8463639495936905, details={'was_impossible': False}), Prediction(uid='751', iid='385', r_ui=4.0, est=3.1807542478219157, details={'was_impossible': False})]


In [7]:
[(pred.uid,pred.iid,pred.est) for pred in predictions[:3]]

[('120', '282', 3.5114147666251547),
 ('882', '291', 3.573872419581491),
 ('535', '507', 4.033583485472447)]

In [8]:
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.49   {'was_impossible': False}


In [9]:
accuracy.rmse(predictions)

RMSE: 0.9467


0.9466860806937948

#### Surprise 주요 모듈 소개

In [11]:
# os 파일 데이터 
import pandas as pd

ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.to_csv('ml-latest-small/ratings_noh.csv', index=False, header=False)

In [16]:
pd.read_csv('ml-latest-small/ratings.csv').head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [14]:
pd.read_csv('ml-latest-small/ratings_noh.csv').head()

,1,1.1,4.0,964982703
0,1,3,4.0,964981247
1,1,6,4.0,964982224
2,1,47,5.0,964983815
3,1,50,5.0,964982931
4,1,70,3.0,964982400


In [12]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5,5))
data = Dataset.load_from_file('ml-latest-small/ratings_noh.csv', reader=reader)

In [18]:
trainset, testset = train_test_split(data,test_size=.25, random_state=0)
algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
round(accuracy.rmse(predictions),4)

RMSE: 0.8682


0.8682

In [24]:
# 판다스 데이터프레임

ratings = pd.read_csv('ml-latest-small/ratings.csv')
reader = Reader(rating_scale=(0.5,5.0))

data = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader)
trainset, testset = train_test_split(data,test_size=.25,random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
print(round(accuracy.rmse(predictions),4))

RMSE: 0.8682
0.8682


## 교차 검증과 하이퍼 파라미터 튜닝

In [25]:
from surprise.model_selection import cross_validate

# 판다스 데이터 프레임 에서 Surprise 데이터 세트로 데이터 로딩
ratings = pd.read_csv('ml-latest-small/ratings.csv')
reader = Reader(rating_scale=(0.5,5))
data = Dataset.load_from_df(ratings[['userId','movieId','rating']],reader)

algo=SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE','MAE'], cv=5, verbose=True)


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8783  0.8764  0.8693  0.8712  0.8708  0.8732  0.0035  
MAE (testset)     0.6741  0.6719  0.6659  0.6716  0.6708  0.6709  0.0027  
Fit time          0.80    0.95    0.91    0.95    0.95    0.91    0.06    
Test time         0.14    0.18    0.13    0.21    0.13    0.16    0.03    


{'test_rmse': array([0.87825225, 0.87642382, 0.86926446, 0.87118128, 0.87075853]),
 'test_mae': array([0.67412216, 0.67191984, 0.66593951, 0.67162754, 0.67079563]),
 'fit_time': (0.8046131134033203,
  0.951977014541626,
  0.9116644859313965,
  0.9509670734405518,
  0.952021598815918),
 'test_time': (0.13658761978149414,
  0.1789999008178711,
  0.13199234008789062,
  0.20806264877319336,
  0.13193559646606445)}

In [ ]:
from surprise.model_selection import GridSearchCV

#최적화할 파라미터 딕셔너리 형태로 지정.
param_grid = {'n_epochs':[20,40,60], 'n_factors':[50,100,200]}

# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse로 수행하도로 그리드서치 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse','mae'], cv=3)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])


## Surprise를 이용한 개인화 영화 추천 시스템 구축

In [28]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5,5))
data_folds = DatasetAutoFolds(ratings_file='ml-latest-small/ratings_noh.csv',reader=reader)
trainset = data_folds.build_full_trainset()

In [29]:
algo = SVD(n_epochs=20, n_factors=50,random_state=0)
algo.fit(trainset)

In [31]:
movies = pd.read_csv('ml-latest-small/movies.csv')
movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count()==0:
    print('없음')
print(movies[movies['movieId']==42])

없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [32]:
uid=str(9)
iid=str(42)
pred = algo.predict(uid,iid,verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


In [35]:
def get_unseen_surprise(ratings,movies,userId):
    seen_movies = ratings[ratings['userId']==userId]['movieId'].tolist()
    total_movies = movies['movieId'].tolist()
    
    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화수:',len(seen_movies), '추천대상 영화수:',len(unseen_movies), '전체 영화수:', len(total_movies))
    return unseen_movies

unseen_movies = get_unseen_surprise(ratings,movies,9)

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742


In [38]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    def sortkey_est(pred):
        return pred.est
    
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions = predictions[:top_n]
    
    top_movie_ids = [int(pred.iid) for pred in top_predictions]
    top_movie_rating = [pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    top_movie_preds = [(id,title,rating) for id, title, rating in zip(top_movie_ids,top_movie_titles, 
                                                                      top_movie_rating)]
    return top_movie_preds

unseen_movies = get_unseen_surprise(ratings,movies,9)
top_movie_preds = recomm_movie_by_surprise(algo,9,unseen_movies,top_n=10)

for top_movie in top_movie_preds:
    print(top_movie[1], ":", top_movie[2])

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957


딥러닝 기반 추천시스템

https://velog.io/@tobigs-recsys/DL-based-Recommender-Systems-%EB%94%A5%EB%9F%AC%EB%8B%9D-%EA%B8%B0%EB%B0%98-%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C


#### 과제_1026_1
lmembers의 상품 구매 데이터를 이용하여 개인맞춤 상품 추천솔루션을 구축 후 다양한 활용 방안을 시현하세요.(금요일 9시 팀별 발표)